# LAB 2

Networks and Systems Security
Week 02
Foundations of Network Security

### Step 3: Sender (Client) – Encrypt & Send
The sender is responsible for encrypting the message and transmitting it securely to the receiver.

code

In [8]:
# sender.py
import socket
import os
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle

# Load recipient's public key
with open("public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# Message to send
message = b"Hello from the secure sender! This is confidential."

# 1. Generate random AES key and IV
aes_key = os.urandom(32) # AES-256
iv = os.urandom(16)

# 2. Encrypt message with AES (CFB mode)
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
encryptor = cipher.encryptor()
encrypted_message = encryptor.update(message) + encryptor.finalize()

# 3. Encrypt AES key with RSA (recipient's public key)
encrypted_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 4. Package: (encrypted_key, iv, encrypted_message)
payload = pickle.dumps((encrypted_key, iv, encrypted_message))

# 5. Send via socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("localhost", 65432))
    s.sendall(payload)
    print("✅ Encrypted message sent!")

✅ Encrypted message sent!


#### Analysis
Here we load the recipients public key and use it to encrypt the message, which is then sent over a TCP connection to the servers IP address and port. Here we use symmetric encryption for the message, and asymetric encryption to keep the symemetric key secure, ensuring only the intended reciever can decrypt the message.

#### Mini reflection
The reason I put the sender in a seporate jupyter notebook is because if it were to be in the same environment, I wouldnt be able to run both the reciever and sender at the same time to send and recieve the message on my local machine. This is an issue I came accross when completing this workshop, therefore i decided to put my sender code here instead.

### Connecting to another device

code

In [7]:
# sender.py
import socket
import os
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle

# Load recipient's public key
with open("friend_public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# Message to send
message = b"Hi Maks! Yes i did!"

# 1. Generate random AES key and IV
aes_key = os.urandom(32) # AES-256
iv = os.urandom(16)

# 2. Encrypt message with AES (CFB mode)
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
encryptor = cipher.encryptor()
encrypted_message = encryptor.update(message) + encryptor.finalize()

# 3. Encrypt AES key with RSA (recipient's public key)
encrypted_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 4. Package: (encrypted_key, iv, encrypted_message)
payload = pickle.dumps((encrypted_key, iv, encrypted_message))

# 5. Send via socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("172.25.128.165", 65432)) #local network ip
    s.sendall(payload)
    print("✅ Encrypted message sent!")

✅ Encrypted message sent!
